# Testando Spark

In [ ]:
import pyspark
# import pandas as pd

In [ ]:
# SparkContext.stop(sc)

In [ ]:
# from pyspark.context import SparkContext
# from pyspark.sql.session import SparkSession

# sc = pyspark.SparkContext(master = "spark://node1:7077",
#                         appName='testando Spark')

# spark = SparkSession(sc)

In [ ]:
# pd_df = pd.read_csv('/home/bigdata/repos/fundamentos-big-data/codes/DNPBA2017.csv')

In [ ]:
# pd_df

In [ ]:
# df = spark.createDataFrame(pd_df)

In [ ]:
df = spark.read.csv('/home/bigdata/repos/fundamentos-big-data/codes/DNPBA2017.csv',
                   header=True)

In [ ]:
df.show(3)

In [ ]:
df.count()

In [ ]:
for column in df.columns:
    df.groupBy(column).count().show()

In [ ]:
df.groupBy('CODMUNRES').count().show()

# Exemplo com HDFS

In [ ]:
df_ = spark.read.csv('hdfs://node1:9000/raw/DNPBA2017.csv', header=True)

In [ ]:
for column in df.columns:
    df_.groupBy(column).count().show()

# Testando ElasticSearch

In [1]:
# !pip install elasticsearch
# !pip install espandas
# !pip install progressbar2
# !pip install es_pandas

In [21]:
import numpy as np
import pandas as pd

from elasticsearch import Elasticsearch
from elasticsearch import helpers
es = Elasticsearch(http_compress=True)

from datetime import datetime

from es_pandas import es_pandas

# adapted from: https://pypi.org/project/es-pandas/

In [3]:
df_pd = pd.read_csv('/home/bigdata/repos/fundamentos-big-data/codes/DNPBA2017.csv')
df_pd = df_pd[["NUMERODN", "CODMUNNASC", "IDADEMAE", "ESCMAEAGR1"]]

In [27]:
df_pd

,NUMERODN,CODMUNNASC,IDADEMAE,ESCMAEAGR1
0,72390242,170700,25,1.0
1,72396320,291110,20,2.0
2,72374657,171890,20,4.0
3,72376065,172100,28,6.0
4,72392109,172100,34,12.0
...,...,...,...,...
203710,77142079,530010,30,8.0
203711,74266407,293180,32,2.0
203712,74266408,293180,23,11.0
203713,74266409,293180,41,11.0


In [ ]:
df_pd.dropna(inplace=True)
df_pd.columns= df_pd.columns.str.lower()

In [16]:
# Information of es cluseter
es_host = 'localhost:9200'
index = 'dnpba_2017'

In [23]:
# create es_pandas instance
ep = es_pandas(es_host)
es = Elasticsearch()

In [18]:
# init template if you want
doc_type = 'nascidosvisvos'
ep.init_es_tmpl(df_pd, doc_type)

New template nascidosvisvos added


In [19]:
# set use_index=True if you want to use DataFrame index as records' _id
ep.to_es(df_pd, index, doc_type=doc_type, use_index=True, thread_count=2, chunk_size=10000)

 98% (201137 of 203715) |############### | Elapsed Time: 0:00:06 ETA:   0:00:00

203715

In [20]:
!curl 'localhost:9200/_cat/indices/'

yellow open nascidosvivos_ba_2017 xgJ4VsEtSlKO7T7tKn8yqA 1 1      0 0   208b   208b
yellow open dnpba_2017            3wXqk-jhSeiRuNE_0O1hzQ 1 1 203715 0 14.5mb 14.5mb
yellow open foo_index             C_Wov6t3TbSdnvST3Q543Q 1 1      0 0   208b   208b
yellow open numerodn              fJXq5vq2S6O5qK7zVIDAjQ 1 1      0 0   208b   208b
yellow open codmunnasc            vimKvxNEQlSQumbUsrrh9g 1 1      0 0   208b   208b
yellow open demo                  8fJBK5yuR2W3jGeWBFEzVQ 1 1 100000 0  3.4mb  3.4mb


##  consultando no elasticsearch

In [28]:
content = {
    "query": {
        "term": {
            "NUMERODN": "72392109"
         }
     },
}

In [29]:
res = es.search(index="dnpba_2017", body=content)

In [30]:
res['hits']['hits']

[{'_index': 'dnpba_2017',
  '_type': '_doc',
  '_id': '4',
  '_score': 1.0,
  '_source': {'NUMERODN': 72392109,
   'CODMUNNASC': 172100,
   'IDADEMAE': 34,
   'ESCMAEAGR1': 12.0}}]